In [1]:
# Install netCDF4 to handle the Xarray dataset engine
!pip install netCDF4 xarray plotly pandas numpy

import xarray as xr
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

print("Libraries imported successfully.")

Libraries imported successfully.


In [2]:
# Defining the path to uploaded dataset
FILE_PATH = '/kaggle/input/xarray-dataset/screening_task (1).nc'

try:
    # Open the dataset; Xarray automatically identifies coordinates and dimensions
    ds = xr.open_dataset(FILE_PATH)
    print("Dataset Structure:")
    print(ds)
    # Verification of the 'forces' variable structure
    print("\nForce Components identified:", ds.Component.values)
except Exception as e:
    print(f"Error loading file: {e}")

Dataset Structure:
<xarray.Dataset> Size: 21kB
Dimensions:    (Element: 85, Component: 30)
Coordinates:
  * Element    (Element) int32 340B 1 2 3 4 5 6 7 8 ... 78 79 80 81 82 83 84 85
  * Component  (Component) object 240B 'Mx_i' 'Mx_j' 'My_i' ... 'x' 'y' 'z'
    Loadcase   object 8B ...
Data variables:
    forces     (Element, Component) float64 20kB ...

Force Components identified: ['Mx_i' 'Mx_j' 'My_i' 'My_j' 'Mz_i' 'Mz_j' 'Vx_i' 'Vx_j' 'Vy_i' 'Vy_j'
 'Vz_i' 'Vz_j' 'ddtheta_x' 'ddtheta_y' 'ddtheta_z' 'ddx' 'ddy' 'ddz'
 'dtheta_x' 'dtheta_y' 'dtheta_z' 'dx' 'dy' 'dz' 'theta_x' 'theta_y'
 'theta_z' 'x' 'y' 'z']


In [3]:
# Member connectivity mapping: element_id : [start_node, end_node]
members_dict = {
    13:[1,11], 22:[11,16], 31:[16,21], 40:[21,26], 49:[26,31], 58:[31,36], 67:[36,41], 76:[41,46], 81:[46,6], # Girder 1
    14:[2,12], 23:[12,17], 32:[17,22], 41:[22,27], 50:[27,32], 59:[32,37], 68:[37,42], 77:[42,47], 82:[47,7], # Girder 2
    15:[3,13], 24:[13,18], 33:[18,23], 42:[23,28], 51:[28,33], 60:[33,38], 69:[38,43], 78:[43,48], 83:[48,8], # Girder 3
    16:[4,14], 25:[14,19], 34:[19,24], 43:[24,29], 52:[29,34], 61:[34,39], 70:[39,44], 79:[44,49], 84:[49,9], # Girder 4
    17:[5,15], 26:[15,20], 35:[20,25], 44:[25,30], 53:[30,35], 62:[35,40], 71:[40,45], 80:[45,50], 85:[50,10] # Girder 5
}

In [4]:
# Node coordinates mapping: node_id : [x, y, z]
nodes_dict = {
    1: [0.0, 0.0, 0.0], 2: [0.0, 0.0, 1.2], 3: [0.0, 0.0, 5.175],
    4: [0.0, 0.0, 9.15], 5: [0.0, 0.0, 10.35], 6: [25.0, 0.0, 0.0],
    7: [25.0, 0.0, 1.2], 8: [25.0, 0.0, 5.175], 9: [25.0, 0.0, 9.15],
    10: [25.0, 0.0, 10.35], 11: [2.7778, 0.0, 0.0], 12: [2.7778, 0.0, 1.2],
    13: [2.7778, 0.0, 5.175], 14: [2.7778, 0.0, 9.15], 15: [2.7778, 0.0, 10.35],
    16: [5.5556, 0.0, 0.0], 17: [5.5556, 0.0, 1.2], 18: [5.5556, 0.0, 5.175],
    19: [5.5556, 0.0, 9.15], 20: [5.5556, 0.0, 10.35], 21: [8.3333, 0.0, 0.0],
    22: [8.3333, 0.0, 1.2], 23: [8.3333, 0.0, 5.175], 24: [8.3333, 0.0, 9.15],
    25: [8.3333, 0.0, 10.35], 26: [11.1111, 0.0, 0.0], 27: [11.1111, 0.0, 1.2],
    28: [11.1111, 0.0, 5.175], 29: [11.1111, 0.0, 9.15], 30: [11.1111, 0.0, 10.35],
    31: [13.8889, 0.0, 0.0], 32: [13.8889, 0.0, 1.2], 33: [13.8889, 0.0, 5.175],
    34: [13.8889, 0.0, 9.15], 35: [13.8889, 0.0, 10.35], 36: [16.6667, 0.0, 0.0],
    37: [16.6667, 0.0, 1.2], 38: [16.6667, 0.0, 5.175], 39: [16.6667, 0.0, 9.15],
    40: [16.6667, 0.0, 10.35], 41: [19.4444, 0.0, 0.0], 42: [19.4444, 0.0, 1.2],
    43: [19.4444, 0.0, 5.175], 44: [19.4444, 0.0, 9.15], 45: [19.4444, 0.0, 10.35],
    46: [22.2222, 0.0, 0.0], 47: [22.2222, 0.0, 1.2], 48: [22.2222, 0.0, 5.175],
    49: [22.2222, 0.0, 9.15], 50: [22.2222, 0.0, 10.35]
}


**Task 1:** SFD and BMD from Xarray dataset


In [5]:
def plot_task_1(ds, n_dict, m_dict):
    # Central longitudinal girder element sequence
    central_ids = [15, 24, 33, 42, 51, 60, 69, 78, 83]
    
    x_coords, vy_vals, mz_vals = [], [], []

    for eid in central_ids:
        # Get start/end nodes and their X-coordinates
        ni_id, nj_id = m_dict[eid]
        xi, xj = n_dict[ni_id][0], n_dict[nj_id][0]
        
        # Select 'forces' array for the current element
        # Based on dataset: Component 1=Vy_i, 2=Mz_i, 4=Vy_j, 5=Mz_j
        element_data = ds.forces.sel(Element=eid)
        
        # Append coordinates and force values (i and j to maintain continuity)
        x_coords.extend([xi, xj])
        vy_vals.extend([float(element_data.isel(Component=1)), float(element_data.isel(Component=4))])
        mz_vals.extend([float(element_data.isel(Component=2)), float(element_data.isel(Component=5))])

    # Create Subplots
    fig = make_subplots(rows=2, cols=1, subplot_titles=("Shear Force Diagram (Vy)", "Bending Moment Diagram (Mz)"))

    # Add SFD Trace
    fig.add_trace(go.Scatter(x=x_coords, y=vy_vals, fill='tozeroy', name='Vy (Shear)', line_color='blue'), row=1, col=1)
    # Add BMD Trace
    fig.add_trace(go.Scatter(x=x_coords, y=mz_vals, fill='tozeroy', name='Mz (Moment)', line_color='red'), row=2, col=1)

    fig.update_layout(height=700, title_text="Task 1: Central Girder Analysis", template="plotly_white")
    fig.show()

# Execution
plot_task_1(ds, nodes_dict, members_dict)

In [6]:
def plot_task_2(ds, n_dict, m_dict, mode='Mz'):
    # Define girder groupings for Task 2
    girders_config = {
        "Girder 1": [13, 22, 31, 40, 49, 58, 67, 76, 81],
        "Girder 2": [14, 23, 32, 41, 50, 59, 68, 77, 82],
        "Girder 3": [15, 24, 33, 42, 51, 60, 69, 78, 83],
        "Girder 4": [16, 25, 34, 43, 52, 61, 70, 79, 84],
        "Girder 5": [17, 26, 35, 44, 53, 62, 71, 80, 85]
    }
    
    fig = go.Figure()
    scale = 0.2  # Force extrusion scale factor
    idx_i, idx_j = (1, 4) if mode == 'Vy' else (2, 5)
    color = 'blue' if mode == 'Vy' else 'red'

    for g_name, e_list in girders_config.items():
        for eid in e_list:
            ni_id, nj_id = m_dict[eid]
            pt_i, pt_j = n_dict[ni_id], n_dict[nj_id]
            
            # Fetch force magnitude from Xarray
            element_data = ds.forces.sel(Element=eid)
            f_i = float(element_data.isel(Component=idx_i)) * scale
            f_j = float(element_data.isel(Component=idx_j)) * scale

            # Draw Girder Frame (Black base line)
            fig.add_trace(go.Scatter3d(x=[pt_i[0], pt_j[0]], y=[pt_i[1], pt_j[1]], z=[pt_i[2], pt_j[2]],
                                       mode='lines', line=dict(color='black', width=2), showlegend=False))

            # Draw Force Extrusion (Colored diagram line)
            fig.add_trace(go.Scatter3d(x=[pt_i[0], pt_j[0]], y=[pt_i[1] + f_i, pt_j[1] + f_j], z=[pt_i[2], pt_j[2]],
                                       mode='lines', line=dict(color=color, width=4), showlegend=False))

            # Draw MIDAS-style vertical connector ribs
            fig.add_trace(go.Scatter3d(x=[pt_i[0], pt_i[0]], y=[pt_i[1], pt_i[1] + f_i], z=[pt_i[2], pt_i[2]],
                                       mode='lines', line=dict(color='lightgray', width=1), showlegend=False))

    fig.update_layout(title=f"Task 2: 3D {mode} Bridge Diagram (MIDAS Style)",
                      scene=dict(aspectmode='data', xaxis_title='X (Span)', yaxis_title='Y (Force)', zaxis_title='Z (Width)'),
                      height=800)
    fig.show()

# Execution - Toggle mode='Vy' for Shear Force or mode='Mz' for Bending Moment
plot_task_2(ds, nodes_dict, members_dict, mode='Mz')